In [1]:
import os
import torch
import argparse
import numpy as np
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
import pickle
from fvcore.common.file_io import PathManager

In [2]:
# Register my amodal datasets 
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_train", {}, "datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json", "datasets/coco/train2014")
# Prepare test datasets 
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
register_coco_instances("amodal_val", {}, "datasets/coco/annotations/COCO_amodal_val2014_with_classes_poly.json", "datasets/coco/val2014")
from detectron2.data import DatasetCatalog
dataset_dicts = DatasetCatalog.get("amodal_train")


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



In [3]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.DATASETS.TRAIN = ("amodal_train",)
cfg.DATASETS.TEST = ("amodal_val",)
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.0005  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
trainer = DefaultTrainer(cfg) 
# import pdb; pdb.set_trace()
trainer.resume_or_load(resume=False)

[02/28 22:26:46 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [02/28 22:26:46 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/28 22:26:46 d2.data.datasets.coco]: Loaded 2276 images in COCO format from datasets/coco/annotations/COCO_amodal_train2014_with_classes_poly.json
[02/28 22:26:46 d2.data.build]: Removed 0 images with no usable annotations. 2276 images left.
[02/28 22:26:46 d2.data.build]: Distribution of instances among all 80 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    person     | 2311         |   bicycle    | 21           |      car      | 279          |
|  motorcycle   | 47           |   airplane   | 65           |      bus      | 92           |
|     train     | 60           |    truck     | 115          |     boat      | 79           |
| traffic light | 13           | fire hydrant | 35           |  

In [4]:
pretrained_path = "~/.torch/fvcore_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
DETECTRON_PATH = os.path.expanduser(pretrained_path)
print(DETECTRON_PATH)

/home/waiyu/.torch/fvcore_cache/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl


In [5]:
with PathManager.open(DETECTRON_PATH, "rb") as f:
    data = pickle.load(f, encoding="latin1")

In [6]:
d = dict(data['model'])

In [7]:
d['roi_heads.visible_mask_head.visible_mask_fcn1.weight'] = d['roi_heads.mask_head.mask_fcn1.weight']
d['roi_heads.visible_mask_head.visible_mask_fcn1.bias'] = d['roi_heads.mask_head.mask_fcn1.bias']
d['roi_heads.visible_mask_head.visible_mask_fcn2.weight'] = d['roi_heads.mask_head.mask_fcn2.weight']
d['roi_heads.visible_mask_head.visible_mask_fcn2.bias'] = d['roi_heads.mask_head.mask_fcn2.bias']
d['roi_heads.visible_mask_head.visible_mask_fcn3.weight'] = d['roi_heads.mask_head.mask_fcn3.weight']
d['roi_heads.visible_mask_head.visible_mask_fcn3.bias'] = d['roi_heads.mask_head.mask_fcn3.bias']
d['roi_heads.visible_mask_head.visible_mask_fcn4.weight'] = d['roi_heads.mask_head.mask_fcn4.weight']
d['roi_heads.visible_mask_head.visible_mask_fcn4.bias'] = d['roi_heads.mask_head.mask_fcn4.bias']
d['roi_heads.visible_mask_head.deconv.weight'] = d['roi_heads.mask_head.deconv.weight']
d['roi_heads.visible_mask_head.deconv.bias'] = d['roi_heads.mask_head.deconv.bias']
d['roi_heads.visible_mask_head.predictor.weight'] = d['roi_heads.mask_head.predictor.weight']
d['roi_heads.visible_mask_head.predictor.bias'] = d['roi_heads.mask_head.predictor.bias']
# Amodal 
d['roi_heads.amodal_mask_head.amodal_mask_fcn1.weight'] = d['roi_heads.mask_head.mask_fcn1.weight']
d['roi_heads.amodal_mask_head.amodal_mask_fcn1.bias'] = d['roi_heads.mask_head.mask_fcn1.bias']
d['roi_heads.amodal_mask_head.amodal_mask_fcn2.weight'] = d['roi_heads.mask_head.mask_fcn2.weight']
d['roi_heads.amodal_mask_head.amodal_mask_fcn2.bias'] = d['roi_heads.mask_head.mask_fcn2.bias']
d['roi_heads.amodal_mask_head.amodal_mask_fcn3.weight'] = d['roi_heads.mask_head.mask_fcn3.weight']
d['roi_heads.amodal_mask_head.amodal_mask_fcn3.bias'] = d['roi_heads.mask_head.mask_fcn3.bias']
d['roi_heads.amodal_mask_head.amodal_mask_fcn4.weight'] = d['roi_heads.mask_head.mask_fcn4.weight']
d['roi_heads.amodal_mask_head.amodal_mask_fcn4.bias'] = d['roi_heads.mask_head.mask_fcn4.bias']
d['roi_heads.amodal_mask_head.deconv.weight'] = d['roi_heads.mask_head.deconv.weight']
d['roi_heads.amodal_mask_head.deconv.bias'] = d['roi_heads.mask_head.deconv.bias']
d['roi_heads.amodal_mask_head.predictor.weight'] = d['roi_heads.mask_head.predictor.weight']
d['roi_heads.amodal_mask_head.predictor.bias'] = d['roi_heads.mask_head.predictor.bias']

In [8]:
newdict = data
newdict['model'] = d

In [9]:
newdict['model']['roi_heads.amodal_mask_head.amodal_mask_fcn1.weight'].shape

(256, 256, 3, 3)

In [10]:
!mkdir pretrained_model
torch.save(newdict, "./pretrained_model/amodal_mask_orcnn_R_50_FPN_3x.pth")
print('saved to {}.'.format("./pretrained_model/amodal_mask_orcnn_R_50_FPN_3x.pth"))

saved to ./pretrained_model/amodal_mask_orcnn_R_50_FPN_3x.pth.
